In [2]:
## imports for custom overlay
from pynq import Overlay
import numpy as np
## imports for baseoverlay HDMI
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
import numpy as np
import asyncio
## Import for DMA alloc
from pynq import allocate



In [12]:
## BASE OVERLAY

base = BaseOverlay("base.bit")
vdma = base.video.axi_vdma

# Set HDMI in and out vars
hdmi_in = base.video.hdmi_in
hdmi_out = base.video.hdmi_out
# initialize IO
hdmi_in.configure(PIXEL_RGB)
hdmi_out.configure(hdmi_in.mode, PIXEL_RGB)

hdmi_in.start()
hdmi_out.start()

In [13]:
# read a frame, allocate a buffer to hold it
frame = hdmi_in.readframe()

in_buffer = allocate(shape=frame.shape, dtype=np.uint8)
out_buffer = allocate(shape=frame.shape, dtype=np.uint8)

in_buffer[:] = frame
hdmi_out.writeframe(in_buffer)

In [14]:
hdmi_out.close()
hdmi_in.close()

In [15]:
## CUSTOM OVERLAY
# Then do the stream overlay for the processing
stream_overlay = Overlay('red.bit')
dma = stream_overlay.redDMA.axi_dma_0

In [7]:
# send input buffer and read the output
def run_kernel():
    dma.sendchannel.transfer(in_buffer)
    dma.recvchannel.transfer(out_buffer)
    dma.sendchannel.wait()
    dma.recvchannel.wait()

In [16]:
run_kernel()


In [17]:
## BASE OVERLAY AGAIN

base = BaseOverlay("base.bit")
vdma = base.video.axi_vdma

# Set HDMI in and out vars
hdmi_in = base.video.hdmi_in
hdmi_out = base.video.hdmi_out
# initialize IO
hdmi_in.configure(PIXEL_RGB)
hdmi_out.configure(hdmi_in.mode, PIXEL_RGB)

hdmi_in.start()
hdmi_out.start()

In [18]:
hdmi_out.writeframe(out_buffer)

In [19]:
hdmi_out.close()
hdmi_in.close()